In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re

import mysql.connector as sql
from sqlalchemy import create_engine

import requests
from bs4 import BeautifulSoup

import time

In [2]:
formated_url="https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start={start:}&ref_=adv_nxt"

title = []
year = []

certificate = []
gross = []
genre = []
run_time = []
rating = []

page_numbers=[0,101,201,301,401,501,601,701,801,901]

for i in page_numbers:
    url=formated_url.format(start=i)
    
    page=requests.get(url)
    
    soup= BeautifulSoup(page.content , "html.parser")
    
    scraped_year = soup.find_all("span" ,class_ ="lister-item-year text-muted unbold")
            
    for YEAR in scraped_year:
        year.append(YEAR.get_text())
            
    scraped_title = soup.find_all("h3" ,class_ ="lister-item-header")
    
    for TITLE in scraped_title:
        title.append(TITLE.get_text().replace("\n",""))
            
    scraped_rating = soup.find_all("strong" )
        
    for RATING in scraped_rating:
        rating.append(RATING.get_text())
        
    scraped_run_time = soup.find_all( class_="runtime" )
    
    for RUNTIME in scraped_run_time:
        run_time.append(RUNTIME.get_text())
        
    scraped_genre= soup.find_all("span",class_="genre")
    
    for GENRE in scraped_genre:
        genre.append(GENRE.text.strip())
                         
    time.sleep(3)
    

## REMOVING UNWANTED TEXT FROM RATING COLUMN

In [3]:
R=rating.copy()

In [4]:
Ratings_df=pd.DataFrame(R)

In [5]:
Ratings_df.replace(["Detailed","User Rating"],np.nan,inplace=True)

In [6]:
Ratings_df.isnull().sum()

0    20
dtype: int64

In [7]:
Ratings_df.dropna(inplace=True)

In [8]:
Ratings_df.reset_index(inplace=True)

In [9]:
Ratings_df.drop(["index"],axis=1,inplace=True)

In [10]:
df=pd.DataFrame()

## CREATING A DATA FRAME FOR TOP 1000 MOVIES FROM IMDB WEBSITE

In [11]:
df["Rank"]=np.arange(1,1001)

In [12]:
df["Title"] = title

In [13]:
df["Year"] = year

In [14]:
df["Rating"] = Ratings_df

In [15]:
df["Run Time (min)"] = run_time

In [16]:
df["Genre"] = genre

In [17]:
df

,Rank,Title,Year,Rating,Run Time (min),Genre
0,1,1.The Shawshank Redemption(1994),(1994),9.3,142 min,Drama
1,2,2.The Godfather(1972),(1972),9.2,175 min,"Crime, Drama"
2,3,3.The Dark Knight(2008),(2008),9.0,152 min,"Action, Crime, Drama"
3,4,4.The Lord of the Rings: The Return of the Kin...,(2003),9.0,201 min,"Action, Adventure, Drama"
4,5,5.Schindler's List(1993),(1993),9.0,195 min,"Biography, Drama, History"
...,...,...,...,...,...,...
995,996,996.Control(2007),(2007),7.6,122 min,"Biography, Drama, Music"
996,997,997.Les yeux sans visage(1960),(1960),7.6,90 min,"Drama, Horror"
997,998,998.Shine(1996),(1996),7.6,105 min,"Biography, Drama, Music"
998,999,999.The Invisible Man(1933),(1933),7.6,71 min,"Horror, Sci-Fi"


In [18]:
df1=df.copy()

## CLEANING COLUMNS

## TITLE COLUMN

In [19]:
def title(text):
    result=re.sub("(^\d+\.)","",str(text))
    result=re.sub("\(\d+\)","",str(result))
    result=re.sub("1,000.","",str(result))
    result=result.strip()
    
    return result

In [20]:
df["Title"]=df["Title"].apply(title)

In [21]:
df

,Rank,Title,Year,Rating,Run Time (min),Genre
0,1,The Shawshank Redemption,(1994),9.3,142 min,Drama
1,2,The Godfather,(1972),9.2,175 min,"Crime, Drama"
2,3,The Dark Knight,(2008),9.0,152 min,"Action, Crime, Drama"
3,4,The Lord of the Rings: The Return of the King,(2003),9.0,201 min,"Action, Adventure, Drama"
4,5,Schindler's List,(1993),9.0,195 min,"Biography, Drama, History"
...,...,...,...,...,...,...
995,996,Control,(2007),7.6,122 min,"Biography, Drama, Music"
996,997,Les yeux sans visage,(1960),7.6,90 min,"Drama, Horror"
997,998,Shine,(1996),7.6,105 min,"Biography, Drama, Music"
998,999,The Invisible Man,(1933),7.6,71 min,"Horror, Sci-Fi"


## YEAR

In [22]:
def year(text):
    result=re.sub("[\(\)]","",str(text))
    result=result.strip()
    
    return result

In [23]:
df["Year"]=df["Year"].apply(year)

In [24]:
df

,Rank,Title,Year,Rating,Run Time (min),Genre
0,1,The Shawshank Redemption,1994,9.3,142 min,Drama
1,2,The Godfather,1972,9.2,175 min,"Crime, Drama"
2,3,The Dark Knight,2008,9.0,152 min,"Action, Crime, Drama"
3,4,The Lord of the Rings: The Return of the King,2003,9.0,201 min,"Action, Adventure, Drama"
4,5,Schindler's List,1993,9.0,195 min,"Biography, Drama, History"
...,...,...,...,...,...,...
995,996,Control,2007,7.6,122 min,"Biography, Drama, Music"
996,997,Les yeux sans visage,1960,7.6,90 min,"Drama, Horror"
997,998,Shine,1996,7.6,105 min,"Biography, Drama, Music"
998,999,The Invisible Man,1933,7.6,71 min,"Horror, Sci-Fi"


## RUN TIME

In [25]:
def run_time(text):
    result=re.sub("min","",str(text))
    result=result.strip()
    return result

In [26]:
df["Run Time (min)"] = df["Run Time (min)"].apply(run_time)

In [27]:
df

,Rank,Title,Year,Rating,Run Time (min),Genre
0,1,The Shawshank Redemption,1994,9.3,142,Drama
1,2,The Godfather,1972,9.2,175,"Crime, Drama"
2,3,The Dark Knight,2008,9.0,152,"Action, Crime, Drama"
3,4,The Lord of the Rings: The Return of the King,2003,9.0,201,"Action, Adventure, Drama"
4,5,Schindler's List,1993,9.0,195,"Biography, Drama, History"
...,...,...,...,...,...,...
995,996,Control,2007,7.6,122,"Biography, Drama, Music"
996,997,Les yeux sans visage,1960,7.6,90,"Drama, Horror"
997,998,Shine,1996,7.6,105,"Biography, Drama, Music"
998,999,The Invisible Man,1933,7.6,71,"Horror, Sci-Fi"


## PUSHING DATA FRAME TO MYSQL

In [28]:
db = sql.connect(host="localhost",
                 user="root",
                 password="1234",
                 database="TOP_1000_IMDB_RATED_MOVIES"
                )

In [29]:
mycursor = db.cursor()

In [30]:
mycursor.execute("select database ()")

In [31]:
mycursor.fetchall()

[('top_1000_imdb_rated_movies',)]

In [32]:
user='root'
password='1234'
host='localhost'
database='TOP_1000_IMDB_RATED_MOVIES'

In [33]:
mycursor.execute("use TOP_1000_IMDB_RATED_MOVIES")

In [34]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

In [35]:
df.to_sql("movies_details",engine,index=False)

1000

## FETCHING DATA FROM MYSQL

In [36]:
mycursor.execute("use TOP_1000_IMDB_RATED_MOVIES")

In [37]:
mycursor.execute("select database ()")

In [38]:
mycursor.fetchall()

[('top_1000_imdb_rated_movies',)]

In [40]:
movies_details_sql=pd.read_sql_query("select * from movies_details",db)

C:\Users\BHAVESH\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [41]:
movies_details_sql

,Rank,Title,Year,Rating,Run Time (min),Genre
0,1,The Shawshank Redemption,1994,9.3,142,Drama
1,2,The Godfather,1972,9.2,175,"Crime, Drama"
2,3,The Dark Knight,2008,9.0,152,"Action, Crime, Drama"
3,4,The Lord of the Rings: The Return of the King,2003,9.0,201,"Action, Adventure, Drama"
4,5,Schindler's List,1993,9.0,195,"Biography, Drama, History"
...,...,...,...,...,...,...
995,996,Control,2007,7.6,122,"Biography, Drama, Music"
996,997,Les yeux sans visage,1960,7.6,90,"Drama, Horror"
997,998,Shine,1996,7.6,105,"Biography, Drama, Music"
998,999,The Invisible Man,1933,7.6,71,"Horror, Sci-Fi"


## EXPORTING DATA TO EXCEL FILE

In [44]:
df.to_csv("TOP_1000_IMDB_RATED_MOVIES_DETAILS.csv",index=False)